<a href="https://colab.research.google.com/github/AwaisAli9012/News-Topic-Classifier-Using-BERT/blob/main/ddh2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate streamlit
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import streamlit as st

In [ ]:
print("Loading AG News dataset...")
dataset = load_dataset("ag_news")

print("Sample from training set:")
print(dataset['train'][0])

Loading AG News dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Sample from training set:
{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


In [ ]:
print("🔧 Loading BERT tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

print("🧽 Tokenizing dataset...")
encoded_dataset = dataset.map(tokenize_function, batched=True)

🔧 Loading BERT tokenizer...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

🧽 Tokenizing dataset...


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
print("🤖 Loading BERT model for classification...")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

model.to(device)

print("📊 Loading evaluation metrics...")
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🤖 Loading BERT model for classification...
✅ Using device: cuda
📊 Loading evaluation metrics...


In [ ]:
print("CUDA available:", torch.cuda.is_available())
training_args = TrainingArguments(
    output_dir="bert-news-classifier-finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",    fp16=torch.cuda.is_available(),
    remove_unused_columns=True,
)


CUDA available: True


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
print("🚀 Starting training...")
trainer.train()


print("📊 Evaluating...")
metrics = trainer.evaluate()
print(f"Test Accuracy: {metrics['eval_accuracy']:.4f}")
print(f"Test F1-Score: {metrics['eval_f1']:.4f}")


print("💾 Saving fine-tuned model...")
model.save_pretrained("bert-news-classifier-finetuned")
tokenizer.save_pretrained("bert-news-classifier-finetuned")

/tmp/ipython-input-72972102.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Starting training...


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.275500,0.231540,0.936842,0.936907
2,0.175700,0.241067,0.946711,0.946731


📊 Evaluating...


Test Accuracy: 0.9467
Test F1-Score: 0.9467
💾 Saving fine-tuned model...


('bert-news-classifier-finetuned/tokenizer_config.json',
 'bert-news-classifier-finetuned/special_tokens_map.json',
 'bert-news-classifier-finetuned/vocab.txt',
 'bert-news-classifier-finetuned/added_tokens.json',
 'bert-news-classifier-finetuned/tokenizer.json')

In [12]:
def predict_topic(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_class].item()
    labels = ["World", "Sports", "Business", "Science/Tech"]
    print(f"Text: {text}")
    print(f"Predicted Topic: {labels[predicted_class]} (Confidence: {confidence:.2f})")
    return labels[predicted_class], confidence


In [13]:
print("\n🔍 Running sample predictions:")
predict_topic("Apple launches new AI-powered iPhone")
predict_topic("Lionel Messi scores winning goal in World Cup final")
predict_topic("Stock market hits new high as tech shares surge")
predict_topic("NASA launches new Mars rover mission")



🔍 Running sample predictions:
Text: Apple launches new AI-powered iPhone
Predicted Topic: Science/Tech (Confidence: 0.99)
Text: Lionel Messi scores winning goal in World Cup final
Predicted Topic: World (Confidence: 0.98)
Text: Stock market hits new high as tech shares surge
Predicted Topic: Business (Confidence: 0.87)
Text: NASA launches new Mars rover mission
Predicted Topic: Science/Tech (Confidence: 1.00)


('Science/Tech', 0.9966422319412231)

In [14]:
@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("bert-news-classifier-finetuned")
    model = AutoModelForSequenceClassification.from_pretrained("bert-news-classifier-finetuned")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return tokenizer, model

# Only launch app if running directly
if __name__ == "__main__":
    st.title("📰 News Topic Classifier")
    st.write("Enter a news headline below to classify its topic using a fine-tuned BERT model.")

    text = st.text_area("Headline:", placeholder="E.g., NASA launches new Mars rover mission...")

    if st.button("Classify"):
        if not text.strip():
            st.warning("Please enter a headline to classify.")
        else:
            tokenizer, model = load_model()
            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
                pred_class = torch.argmax(probs, dim=-1).item()
                confidence = probs[0][pred_class].item()

            labels = ["World", "Sports", "Business", "Science/Tech"]
            st.success(f"**Predicted Topic:** {labels[pred_class]}")
            st.info(f"**Confidence:** {confidence:.2f}")
            st.bar_chart({labels[i]: probs[0][i].item() for i in range(4)})
    else:
        st.info("Enter a headline and click **Classify** to see the result.")

    st.markdown("---")
    st.markdown("Built with 🤗 Transformers & Streamlit | Fine-tuned on AG News dataset")

2025-09-02 09:47:28.094 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 09:47:28.614 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-02 09:47:28.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 09:47:28.618 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 09:47:28.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 09:47:28.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 09:47:28.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 09:47:28.631 Thread 'MainThread': mi